In [1]:
import numpy as np
import pyvista as pv
from collections import defaultdict, deque
import copy

# ----------- Normal vectors estimation functions -----------

def compute_face_normals_and_areas(mesh):
    """
    Computes normal and area for each triangle face in the mesh.

    Returns:
        normals (np.ndarray): An (F, 3) array of unit normal vectors for each face.
        areas (np.ndarray): A (F,) array of areas for each face.

    Method:
        - Extract vertex indices for each triangle face.
        - Compute two edge vectors of the triangle.
        - Use cross product to get the face normal (not normalized).
        - Normalize to get unit normals.
        - Compute triangle area as half of the magnitude of the cross product.
    """
    faces = mesh.faces.reshape((-1, 4))[:, 1:]
    v0 = mesh.points[faces[:, 0]]
    v1 = mesh.points[faces[:, 1]]
    v2 = mesh.points[faces[:, 2]]
    
    cross = np.cross(v1 - v0, v2 - v0)
    areas = 0.5 * np.linalg.norm(cross, axis=1)
    normals = cross / np.maximum(np.linalg.norm(cross, axis=1, keepdims=True), 1e-8)
    return normals, areas

def build_vertex_to_faces_map(mesh):
    """
    Builds a mapping from each vertex index to the list of face indices that include that vertex.

    The same with pyvista's mesh.point_cell_ids()
    Returns:
        defaultdict(list): A dictionary where each key is a vertex index, and the value is 
        a list of face indices (from mesh.faces) that contain the vertex.

    Example:
        {
            0: {1, 5, 10},  # vertex 0 is part of faces 1, 5, and 10
            1: {0, 2},      # vertex 1 is part of faces 0 and 2
            ...
        }    
    """
    vertex_faces = defaultdict(list)
    faces = mesh.faces.reshape((-1, 4))[:, 1:]
    for i, (v1, v2, v3) in enumerate(faces):
        vertex_faces[v1].append(i)
        vertex_faces[v2].append(i)
        vertex_faces[v3].append(i)
    return vertex_faces

def compute_area_weighted_vertex_normals(mesh):
    """
    Computes unit vertex normals by averaging adjacent face normals, weighted by face area.

    Returns:
        vertex_normals (np.ndarray): An (N, 3) array of unit normal vectors for each vertex.

    Method:
        1. Compute face normals and face areas using cross products.
        2. Build a mapping from each vertex to the list of adjacent face indices.
        3. For each vertex:
            - Retrieve adjacent faces.
            - Average their normals, weighted by face area.
            - Normalize the result to obtain a unit normal.
    """
    face_normals, face_areas = compute_face_normals_and_areas(mesh)
    vertex_faces = build_vertex_to_faces_map(mesh)
    num_vertices = mesh.points.shape[0]
    vertex_normals = np.zeros((num_vertices, 3))

    for vi in range(num_vertices):
        adjacent_faces = vertex_faces[vi]
        if not adjacent_faces:
            continue
        areas = face_areas[adjacent_faces]
        normals = face_normals[adjacent_faces]
        weighted_sum = np.sum(normals * areas[:, np.newaxis], axis=0)
        vertex_normals[vi] = weighted_sum / np.linalg.norm(weighted_sum)
    
    return vertex_normals

# -------------------------- Normal vector angle --------------

def normal_vector_angle(n_p, n_q, degrees=False):
    """
    Parameters:
    - n_p, n_q: 3D vectors (can be non-normalized).
    - degrees: If True, return angle in degrees. Otherwise, radians.

    Returns:
    - Angle between vectors
    """
    dot_product = np.dot(n_p, n_q)
    norm_p = np.linalg.norm(n_p)
    norm_q = np.linalg.norm(n_q)

    if norm_p == 0 or norm_q == 0:
        return 0.0

    cos_theta = np.clip(dot_product / (norm_p * norm_q), -1.0, 1.0)
    angle_rad = np.arccos(cos_theta)
    return np.degrees(angle_rad) if degrees else angle_rad

# ----------------------------------------------
def sorting_flat_vertices_with_flatness(f_normals, adjacency):
    """
    Find all flat vertices, compute their flatness (average normal angle to neighbors),
    and return them sorted by flatness (ascending).
    
    Parameters:
        f_normals (np.ndarray): An (M, 3) array containing the normal vectors at each face.
        adjacency (dict): A dictionary mapping each face to a set of its adjacent faces.
        
    Returns:
        List of (vertex_index, flatness_value) tuples, sorted by flatness_value.
    """
    flat_vertices = []
    for vi, neighbors in adjacency.items():
        if not neighbors:
            continue
        # Check if all neighbor angles are below threshold
        angles = [normal_vector_angle(f_normals[vi], f_normals[vj]) for vj in neighbors]
        flatness = sum(angles) / len(angles)
        flat_vertices.append((vi, flatness))
        # if all(angle < threshold for angle in angles):
        #     flatness = sum(angles) / len(angles)
        #     flat_vertices.append((vi, flatness))
        
    # Sort by flatness value (ascending)
    flat_vertices.sort(key=lambda x: x[1])
    return flat_vertices

def sort_faces_by_flatness(face_normals, face_adjacency):
    """
    Compute 'flatness' of each face by averaging the angle to its neighboring face normals,
    then return a list of (face_index, flatness_value) sorted by flatness (ascending).

    Parameters:
        face_normals (np.ndarray): (M, 3) array of face normal vectors.
        face_adjacency (dict[int, set[int]]): adjacency list of faces.

    Returns:
        List[Tuple[int, float]]: list of (face_index, flatness), sorted by flatness.
    """
    flat_faces = []
    for fi, neighbors in face_adjacency.items():
        if not neighbors:
            continue
        angles = [normal_vector_angle(face_normals[fi], face_normals[nj]) for nj in neighbors]
        flatness = sum(angles) / len(angles)
        flat_faces.append((fi, flatness))

    flat_faces.sort(key=lambda x: x[1])  # ascending: flatter faces first
    return flat_faces

from collections import deque
import numpy as np
import copy

def compute_face_normals_from_vertex_normals(faces, v_normals):
    """
    Compute the normal vector of each face in a triangular mesh based on the normal vectors of its vertices.

    Parameters:
        mesh (pyvista.PolyData): A 3D triangular mesh loaded using PyVista.
        v_normals (np.ndarray): An (N, 3) array containing the normal vectors at each vertex.

    Returns:
        np.ndarray: An (M, 3) array containing the normal vector of each face, computed as the average of its three vertices' normals.
    """
    # Average the normal vectors of the three vertices of each face
    face_normals = np.mean(v_normals[faces], axis=1)

    # Normalize the result
    norms = np.linalg.norm(face_normals, axis=1, keepdims=True)
    face_normals = face_normals / np.where(norms == 0, 1, norms)  # avoid division by zero

    return face_normals


def cluster_surface(adj, v_normals, flat_v_sorted, weight, threshold):
    flat_v_sorted_copy = deque(flat_v_sorted)  # Faster .popleft()
    unvisited = set(range(len(v_normals)))
    clusters = []

    def compute_avg_angle(normal, neighbors):
        return np.mean([
            normal_vector_angle(normal, v_normals[nj])
            for nj in neighbors
        ])

    while unvisited:
        while flat_v_sorted_copy:
            start = flat_v_sorted_copy.popleft()
            if start in unvisited:
                break
        else:
            break  # No unvisited nodes left

        cluster = set([start])
        queue = deque([start])
        unvisited.remove(start)

        # Maintain running sum of normals for this cluster
        normals_sum = v_normals[start].copy()

        while queue:
            vi = queue.popleft()
            for vj in adj[vi]:
                if vj not in unvisited:
                    continue

                # Average angle between vj and its neighbors
                avg_angle_vi = compute_avg_angle(v_normals[vi], adj[vj])

                # Compute updated cluster normal
                cluster_normal = normals_sum / np.maximum(np.linalg.norm(normals_sum), 1e-8)

                avg_angle_cluster = compute_avg_angle(cluster_normal, adj[vj])
                proximity = weight * avg_angle_vi + (1 - weight) * avg_angle_cluster

                if proximity < threshold:
                    cluster.add(vj)
                    queue.append(vj)
                    unvisited.remove(vj)
                    normals_sum += v_normals[vj]  # Update running normal sum

        clusters.append(cluster)

    return clusters

def cluster_surface_face_based(face_adjacency, face_normals, flat_f_sorted, weight=0.5, threshold=10.0):
    """
    Cluster faces on a mesh based on surface flatness and adjacency.

    Parameters:
        face_adjacency (dict[int, set[int]]): adjacency list of face indices.
        face_normals (np.ndarray): (M, 3) array of face normal vectors.
        flat_f_sorted (List[Tuple[int, float]]): list of (face_index, flatness), sorted by flatness ascending.
        weight (float): weight for combining local and cluster normal angle.
        threshold (float): max combined angle (in degrees) to include in the cluster.

    Returns:
        List[Set[int]]: list of clusters (each a set of face indices).
    """
    flat_f_sorted_copy = deque(flat_f_sorted)
    unvisited = set(range(len(face_normals)))
    clusters = []

    def compute_avg_angle(normal, neighbors):
        return np.mean([
            normal_vector_angle(normal, face_normals[nj])
            for nj in neighbors
        ])

    while unvisited:
        while flat_f_sorted_copy:
            start, _ = flat_f_sorted_copy.popleft()
            if start in unvisited:
                break
        else:
            break  # No unvisited faces left

        cluster = set([start])
        queue = deque([start])
        unvisited.remove(start)

        # Running sum of normals for this cluster
        normals_sum = face_normals[start].copy()

        while queue:
            fi = queue.popleft()
            for fj in face_adjacency.get(fi, []):
                if fj not in unvisited:
                    continue

                # Average angle of neighbor fj
                avg_angle_local = compute_avg_angle(face_normals[fi], face_adjacency.get(fj, []))

                # Cluster average normal
                cluster_normal = normals_sum / np.maximum(np.linalg.norm(normals_sum), 1e-8)
                avg_angle_cluster = compute_avg_angle(cluster_normal, face_adjacency.get(fj, []))

                # Weighted proximity
                proximity = weight * avg_angle_local + (1 - weight) * avg_angle_cluster

                if proximity < threshold:
                    cluster.add(fj)
                    queue.append(fj)
                    unvisited.remove(fj)
                    normals_sum += face_normals[fj]

        clusters.append(cluster)

    return clusters



def merge_noise_faces(clusters, vertex_normals, adjacency, K=10):
    """
    Merge small (noise) clusters into neighboring real clusters based on normal similarity,
    while avoiding merging adjacent noise clusters together.

    Parameters:
        clusters (List[Set[int]]): List of vertex index sets (each cluster).
        vertex_normals (np.ndarray): (N, 3) array of vertex normals.
        adjacency (dict): {vertex_index: set(neighboring vertex indices)}.
        K (int): Max vertex count for a cluster to be considered noise.

    Returns:
        List[Set[int]]: Updated clusters with noise surfaces merged into real surfaces.
    """
    # Separate clusters
    real_clusters = [c for c in clusters if len(c) >= K]
    noise_clusters = [c for c in clusters if len(c) < K]

    # Precompute normals
    def cluster_avg_normal(cluster):
        normals = [vertex_normals[vi] for vi in cluster]
        summed = np.sum(normals, axis=0)
        return summed / np.linalg.norm(summed) if np.linalg.norm(summed) > 0 else summed

    real_normals = [cluster_avg_normal(c) for c in real_clusters]
    noise_normals = [cluster_avg_normal(c) for c in noise_clusters]

    # Create index-to-cluster maps for fast lookup
    vertex_to_real_cluster = {}
    for idx, cluster in enumerate(real_clusters):
        for v in cluster:
            vertex_to_real_cluster[v] = idx

    merged_indices = set()  # track merged noise cluster indices

    for ni, noise in enumerate(noise_clusters):
        if ni in merged_indices:
            continue
        noise_normal = noise_normals[ni]

        # Find adjacent real clusters only
        neighbor_real_candidates = set()
        for vi in noise:
            for vj in adjacency[vi]:
                if vj in vertex_to_real_cluster:
                    neighbor_real_candidates.add(vertex_to_real_cluster[vj])

        # Find the best matching real cluster by normal vector angle
        best_real_idx = None
        best_angle = float('inf')

        for r_idx in neighbor_real_candidates:
            angle = normal_vector_angle(noise_normal, real_normals[r_idx])
            if angle < best_angle:
                best_real_idx = r_idx
                best_angle = angle

        # Merge into best-matching real cluster
        if best_real_idx is not None:
            real_clusters[best_real_idx].update(noise)
            merged_indices.add(ni)
        # else: discard the noise cluster (could also be kept separately)

    return real_clusters


def merge_noise_faces_face_based(clusters, face_normals, face_adjacency, K=10):
    """
    Merge small (noise) face clusters into neighboring real clusters based on normal similarity,
    while avoiding merging adjacent noise clusters together.

    Parameters:
        clusters (List[Set[int]]): List of face index sets (each cluster).
        face_normals (np.ndarray): (M, 3) array of face normals.
        face_adjacency (dict[int, set[int]]): Adjacency list of faces.
        K (int): Max face count for a cluster to be considered noise.

    Returns:
        List[Set[int]]: Updated clusters with noise faces merged into real face clusters.
    """
    # Separate clusters
    real_clusters = [c for c in clusters if len(c) >= K]
    noise_clusters = [c for c in clusters if len(c) < K]

    # Compute average normal of a cluster
    def cluster_avg_normal(cluster):
        normals = [face_normals[fi] for fi in cluster]
        summed = np.sum(normals, axis=0)
        norm = np.linalg.norm(summed)
        return summed / norm if norm > 0 else summed

    real_normals = [cluster_avg_normal(c) for c in real_clusters]
    noise_normals = [cluster_avg_normal(c) for c in noise_clusters]

    # Map each face to its real cluster (for fast lookup)
    face_to_real_cluster = {}
    for idx, cluster in enumerate(real_clusters):
        for f in cluster:
            face_to_real_cluster[f] = idx

    merged_indices = set()  # to skip already-merged noise clusters

    for ni, noise in enumerate(noise_clusters):
        if ni in merged_indices:
            continue
        noise_normal = noise_normals[ni]

        # Find neighboring real clusters
        neighbor_real_candidates = set()
        for fi in noise:
            for fj in face_adjacency.get(fi, []):
                if fj in face_to_real_cluster:
                    neighbor_real_candidates.add(face_to_real_cluster[fj])

        # Find best-matching real cluster
        best_real_idx = None
        best_angle = float('inf')

        for r_idx in neighbor_real_candidates:
            angle = normal_vector_angle(noise_normal, real_normals[r_idx])
            if angle < best_angle:
                best_real_idx = r_idx
                best_angle = angle

        # Merge noise into the best real cluster
        if best_real_idx is not None:
            real_clusters[best_real_idx].update(noise)
            merged_indices.add(ni)

    return real_clusters



'''
    This code implement the Laplace smoothing algorithm mentioned in the section 2 of the paper 
    "Robust surface segmentation and edge feature lines extraction from fractured
    ragments of relics" 
'''


# ---------- Laplacian Smoothing Functions ----------

def build_vertex_adjacency(faces: np.ndarray) -> dict[int, set[int]]:
    """
    Get adjacency of all vertices in the mesh.

    Parameters:
        faces (np.ndarray): An (N, 3) array where each row contains three vertex indices representing a triangle.

    Returns:
        dict[int, set[int]]: Dictionary mapping each vertex to a set of its adjacent vertices.
    """
    adjacency: dict[int, set[int]] = defaultdict(set)

    for i, j, k in faces:
        adjacency[i].update((j, k))
        adjacency[j].update((i, k))
        adjacency[k].update((i, j))

    return adjacency

def build_face_adjacency(faces):
    """
    Build face adjacency dictionary. Two faces are adjacent if they share exactly two vertices (i.e., share an edge).

    Parameters:
        faces (np.ndarray): An (M, 3) array of triangle vertex indices.

    Returns:
        dict[int, set[int]]: Mapping from face index to set of neighboring face indices.
    """
    edge_to_faces = defaultdict(set)

    # Step 1: Map each edge to the faces that contain it
    for face_idx, (i, j, k) in enumerate(faces):
        edges = [
            tuple(sorted((i, j))),
            tuple(sorted((j, k))),
            tuple(sorted((k, i)))
        ]
        for edge in edges:
            edge_to_faces[edge].add(face_idx)

    # Step 2: Build face adjacency based on shared edges
    face_adjacency = defaultdict(set)
    for edge, face_indices in edge_to_faces.items():
        if len(face_indices) < 2:
            continue  # not a shared edge
        face_indices = list(face_indices)
        for i in range(len(face_indices)):
            for j in range(i + 1, len(face_indices)):
                fi, fj = face_indices[i], face_indices[j]
                face_adjacency[fi].add(fj)
                face_adjacency[fj].add(fi)

    return dict(face_adjacency)

def weighted_laplacian_smoothing(vertices, faces, iterations=1):
    vertices = vertices.copy()
    adjacency = build_vertex_adjacency(faces)

    for _ in range(iterations):
        new_vertices = vertices.copy()
        for i in range(vertices.shape[0]):
            neighbors = list(adjacency[i])
            if not neighbors:
                continue

            neighbors_coords = vertices[neighbors]  # shape: (N_neighbors, 3)
            diffs = neighbors_coords - vertices[i]  # shape: (N_neighbors, 3)
            dists = np.linalg.norm(diffs, axis=1) + 1e-8  # avoid zero division
            weights = 1.0 / dists
            weights /= np.sum(weights)

            weighted_sum = np.dot(weights, diffs)  # shape: (3,)
            new_vertices[i] += weighted_sum

        vertices = new_vertices

    return vertices


def laplacian_smoothing(mesh: pv.PolyData, iterations=3):
    
    # Extract face indices (reshape from PyVista's format)
    faces = mesh.faces.reshape((-1, 4))[:, 1:]  # Drop the leading "3" of each triangle
    adjacency = build_vertex_adjacency(faces)

    points = mesh.points.copy()

    for _ in range(iterations):
        new_points = points.copy()
        for i in range(len(points)):
            neighbors = list(adjacency[i])
            if not neighbors:
                continue
            neighbor_coords = points[neighbors]
            new_points[i] = neighbor_coords.mean(axis=0)
        points = new_points  # update for next iteration

    # Create a new smoothed mesh to avoid altering the original
    smoothed_mesh = pv.PolyData(points, mesh.faces)
    return smoothed_mesh

def compute_mesh_smoothness(vertices, faces): # To see how much a mesh is smooth
    adjacency = build_vertex_adjacency(faces)
    smoothness_vals = []

    for i in range(len(vertices)):
        neighbors = list(adjacency[i])
        if not neighbors:
            continue
        avg_neighbor = np.mean(vertices[neighbors], axis=0)
        smoothness = np.linalg.norm(vertices[i] - avg_neighbor)
        smoothness_vals.append(smoothness)

    return np.mean(smoothness_vals)

In [2]:
def convert_to_builtin_int(obj):
    if isinstance(obj, dict):
        return {convert_to_builtin_int(k): convert_to_builtin_int(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_builtin_int(i) for i in obj]
    elif isinstance(obj, set):
        return {convert_to_builtin_int(i) for i in obj}
    elif isinstance(obj, tuple):
        return tuple(convert_to_builtin_int(i) for i in obj)
    elif isinstance(obj, np.integer):  # Catch np.int64, np.int32, ...
        return int(obj)
    else:
        return obj


In [3]:
import os
import pyvista as pv
import time

time_start = time.time()

mesh = pv.read('../CG_dataset/cuboctahedron_subdivide.obj') 

In [4]:
# faces_1D = mesh.faces
# faces_nD = faces_1D.reshape((-1, 4))[:, 1:]

# smoothed_vertices = weighted_laplacian_smoothing(mesh.points, faces_nD, 3)
# smoothed_mesh = pv.PolyData(smoothed_vertices, faces_1D)

# import gc
# del mesh
# del smoothed_vertices
# gc.collect()

## Laplacian Smoothing

In [5]:
smoothed_mesh = laplacian_smoothing(mesh, iterations=3)

In [6]:
import gc

vertices = smoothed_mesh.points 
faces_1D = smoothed_mesh.faces
faces_nD = smoothed_mesh.faces.reshape((-1, 4))[:, 1:]

del faces_1D
gc.collect()

0

In [7]:
adj = build_vertex_adjacency(faces_nD)
adj = convert_to_builtin_int(adj)

f_adj = build_face_adjacency(faces_nD)
f_adj = convert_to_builtin_int(f_adj)

v_normals = compute_area_weighted_vertex_normals(smoothed_mesh)

# f_normals = compute_face_normals_from_vertex_normals(faces_nD, v_normals)
f_normals, _ = compute_face_normals_and_areas(smoothed_mesh)

# flat_v_sorted = sorting_flat_vertices_with_flatness(v_normals, adj)
# flat_v_sorted = [vertex for vertex, flatness in flat_v_sorted]
# flat_v_sorted = convert_to_builtin_int(flat_v_sorted)

flat_f_sorted = sort_faces_by_flatness(f_normals, f_adj)

## Surface Segmentation 

In [8]:
# clusters = cluster_surface(adj, v_normals, flat_v_sorted, 0.5, 0.55)
clusters = cluster_surface_face_based(f_adj, f_normals, flat_f_sorted, 0.5, 0.4)
#brick1: 0.4
#brick2: 0.5
#brick3: 0.5
#brick4: 0.4
#brick5: 0.4
#brick6: 0.6

In [9]:
# Remove noise faces
# merged = merge_noise_faces(clusters, v_normals, adj, K=10)
merged = merge_noise_faces_face_based(clusters, f_normals, f_adj, K=10)

In [10]:
# colors = [
#     'red', 'green', 'blue', 'yellow', 'magenta', 'cyan',
#     'orange', 'lime', 'deeppink', 'deepskyblue', 'gold',
#     'indigo', 'teal', 'crimson', 'mediumvioletred',
#     'chartreuse', 'orangered', 'darkturquoise',
#     'slateblue', 'darkgoldenrod'
# ]

# plotter = pv.Plotter(shape=(1, 3))

# plotter.subplot(0, 0)
# plotter.add_title("Original")
# plotter.add_mesh(mesh, show_edges=True)

# plotter.subplot(0, 1)
# plotter.add_title("Clusters")
# plotter.add_mesh(smoothed_mesh, show_edges=True)
# for i, cluster in enumerate(clusters):
#     cluster_indices = list(cluster)
#     color = colors[i % len(colors)]
#     plotter.add_mesh(smoothed_mesh.points[cluster_indices], color=color, point_size=10, render_points_as_spheres=True)


# plotter.subplot(0, 2)
# plotter.add_title("Merged")
# plotter.add_mesh(smoothed_mesh, show_edges=True)
# for i, cluster in enumerate(merged):
#     merged_indices = list(cluster)
#     color = colors[i % len(colors)]
#     # if i == 2:
#     #     color = 'white'
#     plotter.add_mesh(smoothed_mesh.points[merged_indices], color=color, point_size=10, render_points_as_spheres=True)

# plotter.link_views()
# plotter.show()

# import gc
# del colors, color, i, plotter, cluster_indices, merged_indices, cluster
# gc.collect()


## Edge Detection

### Edge Points

In [11]:
# def find_cluster_edge_points(clusters, adj):
#     """
#     Identify edge points in each cluster based on adjacency.

#     Args:
#         clusters (List[Set[int]]): List of clusters, each a set of vertex indices.
#         adj (Dict[int, Set[int]]): Vertex adjacency list.

#     Returns:
#         List[Set[int]]: List of sets, each containing the edge vertex indices for that cluster.
#     """
#     cluster_edge_points = []

#     for cluster in clusters:
#         visited = set()
#         edge_points = set()
#         queue = deque()

#         # Initialize BFS with any point in the cluster
#         if not cluster:
#             cluster_edge_points.append(set())
#             continue
#         start = next(iter(cluster))
#         queue.append(start)
#         visited.add(start)

#         while queue:
#             vi = queue.popleft()
#             for vj in adj[vi]:
#                 if vj not in cluster:
#                     edge_points.add(vi)
#                 elif vj not in visited:
#                     visited.add(vj)
#                     queue.append(vj)

#         cluster_edge_points.append(edge_points)

#     return cluster_edge_points


In [12]:
# edge_points_per_cluster = find_cluster_edge_points(merged, adj)
# # Combine all edge points from all clusters
# all_edge_points = set()
# for edge_set in edge_points_per_cluster:
#     all_edge_points.update(edge_set)

In [13]:
# colors = [
#     'red', 'green', 'blue', 'yellow', 'magenta', 'cyan',
#     'orange', 'lime', 'deeppink', 'deepskyblue', 'gold',
#     'indigo', 'teal', 'crimson', 'mediumvioletred',
#     'chartreuse', 'orangered', 'darkturquoise',
#     'slateblue', 'darkgoldenrod'
# ]

# plotter = pv.Plotter(shape=(1, 2))
    
# plotter.subplot(0, 0)
# plotter.add_title("Remove Noise Faces")
# plotter.add_mesh(smoothed_mesh, show_edges=True)
# for i, cluster in enumerate(merged):
#     merged_indices = list(cluster)
#     color = colors[i % len(colors)]
#     plotter.add_mesh(smoothed_mesh.points[merged_indices], color=color, point_size=10)

# plotter.subplot(0, 1)
# # Extract coordinates of all edge points
# plotter.add_title("Edges")
# edge_coords = smoothed_mesh.points[list(all_edge_points)]
# point_cloud = pv.PolyData(edge_coords)
# # Plot edge points
# plotter.add_mesh(point_cloud, color='red', point_size=5)


# plotter.link_views()
# plotter.show()

### Edge Faces

In [14]:
def find_cluster_edge_faces(clusters, face_adjacency):
    """
    Identify edge faces in each cluster based on face adjacency.
    A face is considered an edge face if it has any neighbor outside its cluster.

    Args:
        clusters (List[Set[int]]): List of face clusters (each a set of face indices).
        face_adjacency (Dict[int, Set[int]]): Face adjacency map.

    Returns:
        List[Set[int]]: List of sets, each containing edge face indices per cluster.
    """
    cluster_edge_faces = []

    for cluster in clusters:
        edge_faces = set()

        for face in cluster:
            neighbors = face_adjacency.get(face, set())
            if any(n not in cluster for n in neighbors):
                edge_faces.add(face)

        cluster_edge_faces.append(edge_faces)

    return cluster_edge_faces


In [15]:
# edge_faces_per_cluster = find_cluster_edge_faces(merged, f_adj)
# all_edge_faces = set().union(*edge_faces_per_cluster)

# colors = [
#     'red', 'green', 'blue', 'yellow', 
#     'aliceblue', 'magenta', 'cyan', 'orange', 
#     'lime', 'deeppink', 'darkgray', 'yellowgreen', 
#     'sienna', 'indigo', 'teal', 'purple', 
#     'salmon', 'navy', 'darkgoldenrod', 'black'
# ]

# plotter = pv.Plotter(shape=(2, 2))

# # ========== Subplot 0: Original ==========
# plotter.subplot(0, 0)
# plotter.add_title("Original")
# plotter.add_mesh(mesh, show_edges=True)

# # ========== Subplot 1: Clusters ==========
# plotter.subplot(0, 1)
# plotter.add_title("Clusters")
# plotter.add_mesh(smoothed_mesh, show_edges=True)
# for j, cluster in enumerate(clusters):
#     face_indices = list(cluster)
#     point_indices = np.unique(faces_nD[face_indices].flatten())
#     color = colors[j % len(colors)]
#     plotter.add_mesh(
#         smoothed_mesh.extract_points(point_indices, adjacent_cells=False),
#         color=color,
#         point_size=10,
#         render_points_as_spheres=True
#     )

# # ========== Subplot 2: Merged ==========
# plotter.subplot(1, 0)
# plotter.add_title("Merged")
# plotter.add_mesh(smoothed_mesh, show_edges=True)
# for j, cluster in enumerate(merged):
#     face_indices = list(cluster)
#     point_indices = np.unique(faces_nD[face_indices].flatten())
#     color = colors[j % len(colors)]
#     plotter.add_mesh(
#         smoothed_mesh.extract_points(point_indices, adjacent_cells=False),
#         color=color,
#         point_size=10,
#         render_points_as_spheres=True
#     )

# # === Subplot (1, 1): Edge Faces ===
# plotter.subplot(1, 1)
# plotter.add_title("Edges")
# # plotter.add_mesh(smoothed_mesh, opacity=0.1, show_edges=True)

# # Extract edge faces as cells
# edge_face_mask = np.zeros(smoothed_mesh.n_cells, dtype=bool)
# edge_face_mask[list(all_edge_faces)] = True
# edge_part = smoothed_mesh.extract_cells(edge_face_mask)
# plotter.add_mesh(edge_part, color='orange', show_edges=True)

# # Hiển thị
# plotter.link_views()
# plotter.show()

# # Cleanup (tùy chọn)
# # import gc
# # del colors, i, cluster, face_indices, cell_mask, edge_face_mask, plotter
# # gc.collect()


In [16]:
# edge_faces_per_cluster = find_cluster_edge_faces(merged, f_adj)
# all_edge_faces = set().union(*edge_faces_per_cluster)

# colors = [
#     'red', 'green', 'blue', 'yellow', 
#     'aliceblue', 'magenta', 'cyan', 'orange', 
#     'lime', 'deeppink', 'darkgray', 'yellowgreen', 
#     'sienna', 'indigo', 'teal', 'purple', 
#     'salmon', 'navy', 'darkgoldenrod', 'black'
# ]

# plotter = pv.Plotter(shape=(1, 2))

# # ========== Subplot 0: Original ==========
# plotter.subplot(0, 0)
# plotter.add_text("Surface segmentation", font_size=15)
# plotter.add_mesh(smoothed_mesh)
# for j, cluster in enumerate(clusters):
#     face_indices = list(cluster)
#     point_indices = np.unique(faces_nD[face_indices].flatten())
#     color = colors[j % len(colors)]
#     plotter.add_mesh(
#         smoothed_mesh.extract_points(point_indices, adjacent_cells=False),
#         color=color,
#         point_size=10,
#         render_points_as_spheres=True
#     )

# # ========== Subplot 1: Clusters ==========
# plotter.subplot(0, 1)
# # plotter.add_text("Surface segmentation", font_size=15)
# plotter.add_mesh(smoothed_mesh)
# for j, cluster in enumerate(clusters):
#     face_indices = list(cluster)
#     point_indices = np.unique(faces_nD[face_indices].flatten())
#     color = colors[j % len(colors)]
#     plotter.add_mesh(
#         smoothed_mesh.extract_points(point_indices, adjacent_cells=False),
#         color=color,
#         point_size=10,
#         render_points_as_spheres=True
#     )

# # Hiển thị
# # plotter.link_views()
# plotter.show()

# # Cleanup (tùy chọn)
# # import gc
# # del colors, i, cluster, face_indices, cell_mask, edge_face_mask, plotter
# # gc.collect()


## PCA Post-processing

In [17]:
for j in range(len(merged)):
    surface = merged[j]

    surface_p_id = list()
    for j, f_id in enumerate(surface):
        p_ids = smoothed_mesh.get_cell(f_id).point_ids
        surface_p_id.extend(p_ids)

    surface_p_id = set(surface_p_id)

In [18]:
import numpy as np
from sklearn.decomposition import PCA

real_clusters = list()
noise_clusters = list()

pca_width_threshold = 0.6

for _, f_cluster in enumerate(merged):
    surface_p_id = list()
    for j, f_id in enumerate(f_cluster):
        p_ids = smoothed_mesh.get_cell(f_id).point_ids
        surface_p_id.extend(p_ids)

    surface_p_id = set(surface_p_id)

    pca = PCA(n_components=2)
    transformed = pca.fit_transform(smoothed_mesh.points[list(surface_p_id)])   

    # 2. Chiều dài: dọc theo PC1
    pca_length = transformed[:, 0].max() - transformed[:, 0].min()  

    # 3. Chiều rộng: dọc theo PC2
    pca_width = transformed[:, 1].max() - transformed[:, 1].min()

    if pca_width < pca_width_threshold:
        noise_clusters.append(f_cluster)

    else:
        real_clusters.append(f_cluster)

In [19]:
# axis_colors = ['red', 'blue', 'yellow']

# polydata = pv.PolyData(smoothed_mesh.points[list(surface_p_id)])

# center = polydata.center

# # Chiều dài trục để hiển thị
# scale = 0.3 * smoothed_mesh.length

# # Tạo các Line object cho từng trục (xuất phát từ center)
# lines = [
#     pv.Line(center, center + axis * scale) for axis in axes
# ]

# # Hiển thị cùng với mesh
# plotter = pv.Plotter()

# # Hiển thị lưới smoothed_mesh
# # plotter.add_mesh(smoothed_mesh, show_edges=True, opacity=0.3)
# for i, cluster in enumerate(merged):
#     face_indices = list(cluster)
#     point_indices = np.unique(faces_nD[face_indices].flatten())
#     if i == surface_index:
#         color = colors[i % len(colors)]
#         plotter.add_mesh(
#         smoothed_mesh.extract_points(point_indices, adjacent_cells=False),
#         color=color,
#         point_size=10,
#         render_points_as_spheres=True
#         )   

# # Hiển thị các trục chính màu đỏ
# for line in lines:
#     plotter.add_mesh(line, color='red', line_width=1)

# for i in range(3):
#     plotter.add_mesh(lines[i], color=axis_colors[i])

# plotter.show()

# # NOTE: Project the surfaces onto 2D space and find principal component - easier than working in 3d space

In [20]:
# import numpy as np
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# surface_index = 31
# surface_p_id = list()
# for j, c in enumerate(merged):
#     if j == surface_index:
#         for f_id in c:
#             p_id = smoothed_mesh.get_cell(f_id).point_ids
#             surface_p_id.extend(p_id)
# surface_p_id = set(surface_p_id)

# # 1. Áp dụng PCA
# pca = PCA(n_components=2)
# transformed = pca.fit_transform(smoothed_mesh.points[list(surface_p_id)])

# # 2. Chiều dài: dọc theo PC1
# pca_length = transformed[:, 0].max() - transformed[:, 0].min()

# # 3. Chiều rộng: dọc theo PC2
# pca_width = transformed[:, 1].max() - transformed[:, 1].min()

# print("Chiều dài (PC1):", pca_length)
# print("Chiều rộng (PC2):", pca_width)

# plt.figure(figsize=(6, 6))
# plt.scatter(transformed[:, 0], transformed[:, 1], alpha=0.3,
#             color=colors[surface_index % len(colors)])
# plt.axvline(0, color='red', label='PC1')
# plt.axhline(0, color='blue', label='PC2')

# # Thêm text vào ảnh
# # info_text = f"Length (PC1): {pca_length:.2f}\nWidth (PC2): {pca_width:.2f}"
# # plt.text(-3, 2.8, info_text, fontsize=10, color='black',
# #          bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.3'))

# plt.axis('equal')
# plt.legend()
# plt.title('Surface in PCA')
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.tight_layout()
# plt.show()


In [21]:
# import numpy as np
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# surface_indices = [0, 2, 30, 31]  # 4 surface muốn hiển thị
# # colors = ['blue', 'green', 'orange', 'purple']

# fig, axs = plt.subplots(2, 2, figsize=(12, 12))
# axs = axs.flatten()

# for idx, s_index in enumerate(surface_indices):
#     surface_p_id = []
#     for j, c in enumerate(merged):
#         if j == s_index:
#             for f_id in c:
#                 p_id = smoothed_mesh.get_cell(f_id).point_ids
#                 surface_p_id.extend(p_id)

#     surface_p_id = list(set(surface_p_id))  # loại bỏ trùng lặp

#     if len(surface_p_id) == 0:
#         print(f"⚠️ Surface {s_index} has no points — skipped.")
#         continue

#     # PCA
#     points = smoothed_mesh.points[surface_p_id]
#     pca = PCA(n_components=2)
#     transformed = pca.fit_transform(points)

#     pca_length = transformed[:, 0].max() - transformed[:, 0].min()
#     pca_width = transformed[:, 1].max() - transformed[:, 1].min()

#     ax = axs[idx]
#     ax.scatter(transformed[:, 0], transformed[:, 1], alpha=0.3,
#                color=colors[idx % len(colors)])
#     # ax.axvline(0, color='red', label='PC1')
#     # ax.axhline(0, color='blue', label='PC2')
#     ax.set_title(f'Surface {s_index} in PCA\nLength: {pca_length:.2f}, Width: {pca_width:.2f}')
#     ax.set_xlabel('PC1')
#     ax.set_ylabel('PC2')
#     ax.axis('equal')
#     ax.legend()

# plt.tight_layout()
# plt.show()


In [22]:
def merge_noise_faces_face_based(clusters, face_normals, face_adjacency, real_clusters, noise_clusters):
    """
    Merge small (noise) face clusters into neighboring real clusters based on normal similarity,
    while avoiding merging adjacent noise clusters together.

    Parameters:
        clusters (List[Set[int]]): List of face index sets (each cluster).
        face_normals (np.ndarray): (M, 3) array of face normals.
        face_adjacency (dict[int, set[int]]): Adjacency list of faces.

    Returns:
        List[Set[int]]: Updated clusters with noise faces merged into real face clusters.
    """

    # Compute average normal of a cluster
    def cluster_avg_normal(cluster):
        normals = [face_normals[fi] for fi in cluster]
        summed = np.sum(normals, axis=0)
        norm = np.linalg.norm(summed)
        return summed / norm if norm > 0 else summed

    real_normals = [cluster_avg_normal(c) for c in real_clusters]
    noise_normals = [cluster_avg_normal(c) for c in noise_clusters]

    # Map each face to its real cluster (for fast lookup)
    face_to_real_cluster = {}
    for idx, cluster in enumerate(real_clusters):
        for f in cluster:
            face_to_real_cluster[f] = idx

    merged_indices = set()  # to skip already-merged noise clusters

    for ni, noise in enumerate(noise_clusters):
        if ni in merged_indices:
            continue
        noise_normal = noise_normals[ni]

        # Find neighboring real clusters
        neighbor_real_candidates = set()
        for fi in noise:
            for fj in face_adjacency.get(fi, []):
                if fj in face_to_real_cluster:
                    neighbor_real_candidates.add(face_to_real_cluster[fj])

        # Find best-matching real cluster
        best_real_idx = None
        best_angle = float('inf')

        for r_idx in neighbor_real_candidates:
            angle = normal_vector_angle(noise_normal, real_normals[r_idx])
            if angle < best_angle:
                best_real_idx = r_idx
                best_angle = angle

        # Merge noise into the best real cluster
        if best_real_idx is not None:
            real_clusters[best_real_idx].update(noise)
            merged_indices.add(ni)

    return real_clusters

In [23]:
merged = merge_noise_faces_face_based(merged, f_normals, f_adj, real_clusters, noise_clusters)

In [24]:
edge_faces_per_cluster = find_cluster_edge_faces(merged, f_adj)
all_edge_faces = set().union(*edge_faces_per_cluster)

colors = [
    'red', 'green', 'blue', 'yellow', 
    'aliceblue', 'magenta', 'cyan', 'orange', 
    'lime', 'deeppink', 'darkgray', 'yellowgreen', 
    'sienna', 'indigo', 'teal', 'purple', 
    'salmon', 'navy', 'darkgoldenrod', 'black'
]

plotter = pv.Plotter(shape=(1, 2))

# # ========== Subplot 0: Original ==========
plotter.subplot(0, 0)
# plotter.add_title("Original")
plotter.add_mesh(mesh, show_edges=True, color='aliceblue')

# # ========== Subplot 1: Clusters ==========
# plotter.subplot(0, 1)
# plotter.add_title("Clusters")
# plotter.add_mesh(smoothed_mesh, show_edges=True)
# for j, cluster in enumerate(clusters):
#     face_indices = list(cluster)
#     point_indices = np.unique(faces_nD[face_indices].flatten())
#     color = colors[j % len(colors)]
#     plotter.add_mesh(
#         smoothed_mesh.extract_points(point_indices, adjacent_cells=False),
#         color=color,
#         point_size=10,
#         render_points_as_spheres=True
#     )

# ========== Subplot 2: Merged ==========
# plotter.subplot(0, 0)
# plotter.add_text("Long and thin faces merging", font_size=15)
# plotter.add_mesh(smoothed_mesh, show_edges=True)
# for j, cluster in enumerate(merged):
#     face_indices = list(cluster)
#     point_indices = np.unique(faces_nD[face_indices].flatten())
#     color = colors[j % len(colors)]
#     plotter.add_mesh(
#         smoothed_mesh.extract_points(point_indices, adjacent_cells=False),
#         color=color,
#         point_size=10,
#         render_points_as_spheres=True
#     )

# === Subplot (1, 1): Edge Faces ===
plotter.subplot(0, 1)
# plotter.add_text("Edges", font_size=12)
# plotter.add_mesh(smoothed_mesh, opacity=0.1, show_edges=True)

# Extract edge faces as cells
edge_face_mask = np.zeros(smoothed_mesh.n_cells, dtype=bool)
edge_face_mask[list(all_edge_faces)] = True
edge_part = smoothed_mesh.extract_cells(edge_face_mask)
plotter.add_mesh(edge_part, color='orange', show_edges=True)

# Hiển thị
plotter.link_views()
plotter.show(window_size=(2000, 1000))

# Cleanup (tùy chọn)
# import gc
# del colors, i, cluster, face_indices, cell_mask, edge_face_mask, plotter
# gc.collect()


Widget(value='<iframe src="http://localhost:39751/index.html?ui=P_0x7762ed2cc200_0&reconnect=auto" class="pyvi…

In [25]:
time_end = time.time()

print(f"Running times (s): {time_end - time_start}")

Running times (s): 5.7506797313690186


In [26]:
mesh

PolyData (0x7762b85c0820)
  N Cells:    20480
  N Points:   10242
  N Strips:   0
  X Bounds:   -1.000e+00, 1.000e+00
  Y Bounds:   -1.000e+00, 1.000e+00
  Z Bounds:   -1.000e+00, 1.000e+00
  N Arrays:   1

In [27]:
# import nbformat

# # 📌 BƯỚC 1: Đặt tên file notebook và file xuất ra
# notebook_file = "RobustSurfaceSeg_EdgeExtract.ipynb"     # 👉 đổi thành tên notebook của bạn
# output_file = "extract_code_from_ipynb.py"      # 👉 tên file .py để lưu code

# # 📌 BƯỚC 2: Đọc file notebook
# with open(notebook_file, "r", encoding="utf-8") as f:
#     nb = nbformat.read(f, as_version=4)

# # 📌 BƯỚC 3: Giả sử ô hiện tại đang chạy là ô cuối cùng (có thể điều chỉnh nếu cần)
# current_cell_index = len(nb.cells) - 1

# # 📌 BƯỚC 4: Lấy code từ tất cả các cell code phía trên
# code_above = ""
# for i in range(current_cell_index):
#     cell = nb.cells[i]
#     if cell.cell_type == "code":
#         code_above += cell.source + "\n\n"

# # 📌 BƯỚC 5: Lưu vào file .py
# with open(output_file, "w", encoding="utf-8") as f:
#     f.write(code_above)

# print(f"✅ Đã lưu mã từ các cell phía trên vào file: {output_file}")
